In [1]:
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import numpy as np

# Especifique o caminho do arquivo Excel
caminho_do_arquivo = 'data/dataset.xlsx'

In [2]:
# Substitua 'nome_da_tabela' pelo nome da tabela específica que você deseja importar
nome_tabela_1 = 'DATA SET 1'

# Carregando a tabela específica em um DataFrame
try:
    dados = pd.read_excel(caminho_do_arquivo, sheet_name=nome_tabela_1)
    print(f"Dados da tabela '{nome_tabela_1}' importados com sucesso.")
    # Exiba o DataFrame ou realize outras operações com os dados, se necessário
except Exception as e:
    print(f"Erro ao importar dados da tabela '{nome_tabela_1}': {str(e)}")

Dados da tabela 'DATA SET 1' importados com sucesso.


In [4]:
# Função para remover outliers usando o método IQR
def remove_outliers(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return data[(data[column] >= lower_bound) & (data[column] <= upper_bound)]

# Remover outliers das colunas relevantes
dados_filtered = remove_outliers(dados, 'Product Pellets')
dados_filtered = remove_outliers(dados_filtered, 'DDRS Rejects/Feed')
dados_filtered = remove_outliers(dados_filtered, 'SDRS Rejects/Feed')

# Extraindo as colunas relevantes do DataFrame filtrado
dates = dados_filtered['Date']
pellets = dados_filtered['Product Pellets']
ddrs_rejects = dados_filtered['DDRS Rejects/Feed']
sdrs_rejects = dados_filtered['SDRS Rejects/Feed']

# Criando DataFrame para Plotly
df_plotly = pd.DataFrame({'Date': dates, 'Pellets': pellets, 'DDRS Rejects': ddrs_rejects * 100, 'SDRS Rejects': sdrs_rejects * 100})

# Criando gráfico interativo com Plotly Express
fig = px.scatter(df_plotly, x='Pellets', y=['DDRS Rejects', 'SDRS Rejects'],
                 labels={'variable': 'Select view', 'value': 'Percentage'},
                 title='Evolution of DDRS and SDRS in relation to Product Pellets (No Outliers)',
                 hover_data=['Date'], trendline='ols')

# Exibindo o gráfico interativo
fig.show()


In [5]:
dados_filtered.head()

,Date,Product Pellets,DDRS Rejects/Feed,SDRS Rejects/Feed
0,2019-12-01,696.137593,0.113790,0.195691
1,2019-12-02,728.049589,0.139619,0.208134
2,2019-12-03,685.259547,0.108996,0.191460
3,2019-12-04,720.261113,0.109885,0.176185
6,2019-12-07,711.740942,0.108513,0.195419


In [3]:
# Substitua 'nome_da_tabela' pelo nome da tabela específica que você deseja importar
nome_tabela_2 = 'DATA SET 2'

# Carregando a tabela específica em um DataFrame
try:
    df = pd.read_excel(caminho_do_arquivo, sheet_name=nome_tabela_2)
    print(f"Dados da tabela '{nome_tabela_2}' importados com sucesso.")
    # Exiba o DataFrame ou realize outras operações com os dados, se necessário
except Exception as e:
    print(f"Erro ao importar dados da tabela '{nome_tabela_2}': {str(e)}")

# Título da tabela "Bed height measured form grate"


Dados da tabela 'DATA SET 2' importados com sucesso.


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2702 entries, 0 to 2701
Data columns (total 25 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Time [sec]                    2702 non-null   int64  
 1   Time [min]                    2702 non-null   float64
 2   Zone                          2612 non-null   object 
 3   Unnamed: 3                    2702 non-null   int64  
 4   pressure (mbar)               2702 non-null   int64  
 5   pressure (mbar).1             567 non-null    float64
 6   Flow rate [Nm³/h] with Error  794 non-null    object 
 7   Flow rate [Nm³/h]             2702 non-null   float64
 8   T SP above                    2702 non-null   float64
 9   T PV above                    2702 non-null   float64
 10  Bed h 40 cm                   2702 non-null   float64
 11  Bed h 32 cm                   2702 non-null   float64
 12  Bed h 26 cm                   2702 non-null   float64
 13  Bed

In [5]:
df.columns

Index(['Time [sec]', 'Time [min]', 'Zone', 'Unnamed: 3', 'pressure (mbar)',
       'pressure (mbar).1', 'Flow rate [Nm³/h] with Error',
       'Flow rate [Nm³/h]', 'T SP above', 'T PV above', 'Bed h 40 cm',
       'Bed h 32 cm', 'Bed h 26 cm', 'Bed h 18 cm', 'Bed h 10 cm', 'Bed Tm',
       'Bed Tspread [K]}', 'T SP below [°C]', 'T PV below [°C]', 'O2 dry [%]',
       'O2 wet [%]', 'SO2 [mg/m³]', 'Nox [mg/m³]', 'CO2 [%]', 'CO [mg/m³]'],
      dtype='object')

In [17]:
# Vamos criar uma função para aplicar a lógica de preenchimento
def custom_fillna(column):
    if column.name == 'pressure (mbar).1':
        # Preencher com a média
        return column.fillna(column.mean())
    elif column.name == 'Flow rate [Nm³/h] with Error':
        # Preencher com 0
        return column.fillna(0)
    else:
        # Manter outros valores como estão
        return column

# Aplicar a função para preenchimento personalizado
df = df.apply(custom_fillna)

# Se você quiser substituir '-' por NaN antes de aplicar as condições acima, pode fazer algo assim:
df.replace('-', pd.NA, inplace=True)

In [18]:
df.describe()

,Time [sec],Time [min],Unnamed: 3,pressure (mbar),pressure (mbar).1,Flow rate [Nm³/h] with Error,Flow rate [Nm³/h],T SP above,T PV above,Bed h 40 cm,...,Bed Tm,Bed Tspread [K]},T SP below [°C],T PV below [°C],O2 dry [%],O2 wet [%],SO2 [mg/m³],Nox [mg/m³],CO2 [%],CO [mg/m³]
count,2702.000000,2702.000000,2702.000000,2702.000000,2702.000000,2702.000000,2702.000000,2702.000000,2702.000000,2702.000000,...,2702.000000,2702.000000,2702.000000,2702.000000,2702.000000,2702.000000,2702.000000,2702.000000,2702.000000,2702.000000
mean,1350.500000,22.508333,-94.559585,-3.936714,18.765432,36.036655,248.024633,480.422644,714.292893,824.497875,...,654.078755,399.890652,30.862324,290.452971,20.236644,16.224187,109.659970,27.140893,2.961753,43.049830
std,780.144538,13.002409,106.768400,38.855689,9.314446,108.325679,120.096832,498.747887,394.121967,447.992733,...,460.903729,276.737071,92.833329,277.320291,0.599184,1.786620,257.130364,40.492801,2.550235,112.402882
min,0.000000,0.000000,-100.000000,-42.000000,-30.000000,0.000000,89.385007,0.000000,26.415900,8.101600,...,13.448700,22.279400,0.000000,18.827300,17.208700,9.309600,0.000000,-2.702181,0.079887,1.708416
25%,675.250000,11.254167,-100.000000,-40.000000,18.765432,0.000000,152.562786,0.000000,354.894450,397.081250,...,153.019565,148.766325,0.000000,59.409150,19.783900,15.192400,0.000000,0.000000,0.205597,9.002389
50%,1350.500000,22.508333,-100.000000,-30.000000,18.765432,0.000000,199.148902,350.000000,798.886950,1006.222700,...,639.461820,323.862750,0.000000,197.443250,20.254100,15.928400,0.000000,9.735652,2.665910,12.375371
75%,2025.750000,33.762500,-100.000000,40.000000,18.765432,0.000000,310.312757,1028.156275,1027.959900,1216.865100,...,1158.351085,643.674700,0.000000,401.371000,20.861700,16.897700,26.595537,28.966484,5.624359,17.799432
max,2701.000000,45.016667,2000.000000,40.000000,40.000000,384.340700,522.534169,1230.000000,1232.490700,1312.869500,...,1263.044440,961.663800,310.000000,893.456400,20.905100,20.586800,1126.600399,168.844119,7.887350,596.030789


In [9]:
# Selecionando as colunas relevantes
cols_to_plot = ['Time [min]', 'pressure (mbar)', 'Flow rate [Nm³/h]', 'T SP above', 'T PV above',
                'Bed h 40 cm', 'Bed h 32 cm', 'Bed h 26 cm', 'Bed h 18 cm', 'Bed h 10 cm',
                'Bed Tm', 'Bed Tspread [K]}', 'T SP below [°C]', 'T PV below [°C]',
                'O2 dry [%]', 'O2 wet [%]', 'SO2 [mg/m³]', 'Nox [mg/m³]', 'CO2 [%]', 'CO [mg/m³]']

df_selected = df[cols_to_plot]

# Criando o gráfico interativo
fig = px.line(df_selected, x='Time [min]', y=cols_to_plot, title='Time Series Visualization of Process Variables',
              labels={'value': 'Value', 'variable': 'Variable'},
              line_shape='linear')

# Exibindo o gráfico
fig.show()
